<a href="https://colab.research.google.com/github/maxGrigorenko/DistributionClassifier/blob/maxGrigorenko%2Ffirst_part/src/experiments_first_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Классификация распределений. Первая часть проекта
## 3. Построение множества $\mathscr{A}$ и оценка ошибки


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import product


from graph_common_functions import *
from distibution_functions import *

In [2]:
sigma = 1
beta = (1/2) ** 0.5

In [3]:
class DominationNubmerStats:
    def __init__(self, value):
        self.value = value
        self.normal_counter = 0
        self.laplace_counter = 0

def get_stats(n, d, number_of_experiments=1000, alpha=0.05, verbose=True):
    domination_number_stats = [DominationNubmerStats(v) for v in range(1, 100+1)]

    if verbose:
        experiments = tqdm(range(number_of_experiments))
    else:
        experiments = range(number_of_experiments)
    for t in experiments:
        normal_array = generate_normal(sigma, n)
        g = distance_graph_constructor(normal_array, d)
        dominating_number = g.compute_dominating_number(d)
        inx = dominating_number - 1
        domination_number_stats[inx].normal_counter += 1

        laplace_array = generate_laplace(beta, n)
        g = distance_graph_constructor(laplace_array, d)
        dominating_number = g.compute_dominating_number(d)
        inx = dominating_number - 1
        domination_number_stats[inx].laplace_counter += 1

    return domination_number_stats

def construct_A(n, d, number_of_experiments=1000, alpha=0.05, verbose=True, from_err2=False):
    A = []
    domination_number_stats = get_stats(n, d, number_of_experiments=number_of_experiments, alpha=alpha, verbose=verbose)
    for d in domination_number_stats:
        if (d.laplace_counter + d.normal_counter) > 0:
            err1 = d.laplace_counter/(d.laplace_counter + d.normal_counter)
            err2 = d.normal_counter/(d.laplace_counter + d.normal_counter)

            if from_err2:
                if err2 <= alpha:
                    A.append(d.value)

            else:
                if err1 <= alpha:
                    A.append(d.value)

    return A

In [4]:
for n in range(5, 101, 5):
    best_A = []
    best_d = 0.0
    m = 0
    for d_inx in range(1, 40, 3):
        d = d_inx/10
        A = construct_A(n=n, d=d, number_of_experiments=1000, alpha=0.05, verbose=False)
        if len(A) >= m:
            best_A = A
            best_d = d
            m = len(A)

    print(f'n={n}, best_d={best_d}, best_A={best_A}')

n=5, best_d=3.7, best_A=[]
n=10, best_d=0.4, best_A=[8]
n=15, best_d=0.4, best_A=[9]
n=20, best_d=3.7, best_A=[]
n=25, best_d=0.1, best_A=[7, 21]
n=30, best_d=3.7, best_A=[]
n=35, best_d=0.4, best_A=[3]
n=40, best_d=0.1, best_A=[24, 25]
n=45, best_d=0.1, best_A=[25]
n=50, best_d=0.1, best_A=[25, 26]
n=55, best_d=3.7, best_A=[]
n=60, best_d=0.7, best_A=[2]
n=65, best_d=3.7, best_A=[]
n=70, best_d=3.7, best_A=[]
n=75, best_d=1.6, best_A=[1]
n=80, best_d=1.6, best_A=[1]
n=85, best_d=1.6, best_A=[1]
n=90, best_d=0.1, best_A=[29]
n=95, best_d=1.0, best_A=[2]
n=100, best_d=3.7, best_A=[]


Видно, что даже при рассмотрении множества различных d, при определeнных n, множество $\mathscr{A}$ бывает путстым, что говорит о слабости критерия, при установке конкретного d (например 1.6) при большинстве значений n множество  $\mathscr{A}$ будет пустым.   

В прошлом блокноте мы увидели, что при значениях d около 3 часто значение доминирующего числа у распределения Лапласа может выделяться. Немного отходя от задания, протестируем построение $\mathscr{A}$ относительно другой ошибки (то есть в случае, если распрделения поменять местами)

In [5]:
for n in range(5, 101, 5):
    best_A = []
    best_d = 0.0
    m = 0
    for d_inx in range(1, 40, 3):
        d = d_inx/10
        A = construct_A(n=n, d=d, number_of_experiments=500, alpha=0.05, verbose=False, from_err2=True)
        if len(A) >= m:
            best_A = A
            best_d = d
            m = len(A)

    print(f'n={n}, best_d={best_d}, best_A={best_A}')

n=5, best_d=3.7, best_A=[2]
n=10, best_d=3.7, best_A=[2]
n=15, best_d=3.7, best_A=[2, 3]
n=20, best_d=1.0, best_A=[1, 5]
n=25, best_d=1.6, best_A=[4, 5]
n=30, best_d=0.1, best_A=[8, 9, 21]
n=35, best_d=0.4, best_A=[3, 9, 10]
n=40, best_d=3.7, best_A=[2, 3]
n=45, best_d=2.2, best_A=[3, 4]
n=50, best_d=3.4, best_A=[2, 3]
n=55, best_d=1.9, best_A=[3, 4]
n=60, best_d=0.4, best_A=[10, 12, 13]
n=65, best_d=3.7, best_A=[2, 3]
n=70, best_d=1.6, best_A=[1, 4, 5]
n=75, best_d=0.4, best_A=[10, 11, 12]
n=80, best_d=0.4, best_A=[10, 11, 12]
n=85, best_d=1.9, best_A=[3, 4]
n=90, best_d=1.6, best_A=[3, 4, 5]
n=95, best_d=0.4, best_A=[10, 11, 13]
n=100, best_d=0.4, best_A=[10, 11, 12]


В частности при d = 3.5

In [14]:
d = 3.5
for n in range(5, 101, 5):
    A = construct_A(n=n, d=d, number_of_experiments=3000, alpha=0.05, verbose=False, from_err2=True)
    print(f'n={n}, A={A}')

n=5, A=[2]
n=10, A=[2]
n=15, A=[2, 3]
n=20, A=[2]
n=25, A=[2]
n=30, A=[2]
n=35, A=[2, 3]
n=40, A=[2, 3]
n=45, A=[2]
n=50, A=[2]
n=55, A=[2, 3]
n=60, A=[2]
n=65, A=[2, 3]
n=70, A=[2, 3]
n=75, A=[2, 3]
n=80, A=[2, 3]
n=85, A=[2, 3]
n=90, A=[2, 3]
n=95, A=[2, 3]
n=100, A=[2, 3]


Это хороший критерий. Как мы видели в прошлом блокноте, при d=3.5 характеристика графа нормального распределения почти всегда 1, а у графа экспоненциального распределения в значительной части случаев (при n > 30 обычно более чем в 15% случаев) значение характеристики больше 1.